In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import gensim
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
import pandas as pd
import joblib

In [1]:
import pandas as pd
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
import joblib

# Read the CSV file
df = pd.read_csv('/content/drive/MyDrive/Complete_Resume_Screening/Resume_Screening_Project/allJobs.csv')

# Extract the last column
last_column_name = df.columns[-1]
last_column_values = df[last_column_name].head(1000)  # Take only the first 5000 rows

# Tokenize the rows in the last column
sent = [row.split() for row in last_column_values]

# Detect phrases
phrases = Phrases(sent, min_count=30, progress_per=10000)
bigram = Phraser(phrases)

frozen_model = phrases.freeze()
frozen_model.save("/content/drive/MyDrive/Complete_Resume_Screening/Resume_Screening_Project/Phrases_split.pkl")

# Transform the sentences to include phrases
sentences = [bigram[sentence] for sentence in sent]

# Initialize and train the Word2Vec model
w2v_model = Word2Vec(
    min_count=10,
    window=3,
    vector_size=300,
    sample=6e-5,
    alpha=0.03,
    min_alpha=0.0007,
    negative=20
)

# Build vocabulary
w2v_model.build_vocab(sentences)

# Saving the built vocabulary locally
with open('vocabulary.csv', 'w') as f:
    for word in w2v_model.wv.key_to_index.keys():
        f.write("%s\n" % word)

# Train the model
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

# Save the model
path = "/content/drive/MyDrive/Complete_Resume_Screening/Resume_Screening_Project/word2vec_model.model"
w2v_model.save(path)

# Load the model (if needed)
# model = joblib.load(path)



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
model = Word2Vec.load(path)

# Example similarity check
#print(model.wv.similarity('data', 'machine_learning'))

def to_la(L):
  k=list(L)
  l=np.array(k)
  return l.reshape(-1, 1)

def cos(A, B):
  dot_prod=np.matmul(A,B.T)
  norm_a=np.reciprocal(np.sum(np.abs(A)**2,axis=-1)**(1./2))
  norm_b=np.reciprocal(np.sum(np.abs(B)**2,axis=-1)**(1./2))
  norm_a=to_la(norm_a)
  norm_b=to_la(norm_b)
  k=np.matmul(norm_a,norm_b.T)
  return list(np.multiply(dot_prod,k))



In [ ]:
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cos

def check(pdftext, skills, l2, w2v_model1, phrases, pattern):
    text = pdftext
    text = re.sub(r'[^\x00-\x7f]', r' ', text)
    text = text.lower()
    text = re.sub(r"\\\|,|/|:|\)|\(", " ", text)
    t2 = text.split()
    l_2 = l2.copy()
    match = list(set(re.findall(pattern, text)))
    sentences = phrases[t2]
    resume_skills_dict = {}
    res_jdskill_intersect = list(set(sentences).intersection(set(l_2)))

    if len(match) != 0:
        for k in match:
            k = k.replace(' ', '_')
            resume_skills_dict[k] = 1
            try:
                l_2.remove(k)
            except:
                continue

    l6 = list(set(l_2).intersection(skills['0']))
    l6_minus_skills = list(set(l_2).difference(skills['0']))

    for i in l6_minus_skills:
        resume_skills_dict[i] = 0

    if len(l6) == 0:
        return resume_skills_dict

    l4 = list(set(sentences).intersection(skills['0']))
    arr1 = np.array([w2v_model1[i] for i in l6])
    arr2 = np.array([w2v_model1[i] for i in l4])
    similarity_values = cos(arr1, arr2)
    count = 0

    for i in similarity_values:
      k=list(filter(lambda x: x<0.38, list(i)))
      if(len(k)==len(i)):
        resume_skills_dict[l6[count]]=0
      else:
        resume_skills=[s for s in range(len(i)) if(i[s])>0.38]
        resume_skills_dict[l6[count]]=1
      count+=1

    return resume_skills_dict


In [ ]:
def Convert(string):
    li = list(string.split())
    return list(set(li))

def preprocess(string):
  string = string.replace(",",' ')
  string= string.replace("'",' ')
  string = Convert(string)
  return string

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import csv
import re

resume_strs = []
names = []

with open('/content/drive/MyDrive/Complete_Resume_Screening/Resume_Screening_Project/candidates.csv', 'r') as csvfile:
  csv_reader = csv.reader(csvfile)
  for row in csv_reader:
    names.append(row[0])
    resume_strs.append(row[4])


def calculate_weighted_similarity_score(matching_results, required_skills):
    matching_score = 0
    total_weight = 0

    for skill, weight in required_skills.items():
        total_weight += weight
        if skill in matching_results:
            matching_score += matching_results[skill] * weight

    # Calculate the weighted similarity score
    similarity_score = matching_score / total_weight

    return similarity_score


def string_to_dict(list_of_lines):
    # Split the string into key-value pairs

    # Initialize an empty dictionary
    result_dict = {}

    # Iterate over each pair
    for line in list_of_lines:
      if ':' in line:
            # Split the pair into key and value
            key, value = line.split(':')

            # Convert the value to the appropriate type (e.g., int)
            value = int(value)

            # Add the key-value pair to the dictionary
            result_dict[key] = value
      else:
            print(f"Warning: Skipping line '{line.strip()}' due to missing colon.")

    return result_dict



def get_candidate_compatability(required_skills,pdf_text):
  pdftext = pdf_text
  jd_skills = required_skills
  jd_skills = jd_skills.replace('\n',', ')
  jd_skills = re.sub(r'\d+', '', jd_skills)
  jd_skills = jd_skills.replace(':','')



  skills = pd.read_csv('/content/drive/MyDrive/Complete_Resume_Screening/Resume_Screening_Project/vocabulary.csv', quoting=csv.QUOTE_NONE, delimiter = '/t' ,header=None, names=['0'])

  mapper = {}
  underscore=[]

  jd_skills=" ".join(jd_skills.strip().split())
  jd_skills=jd_skills.replace(', ',',')
  pattern=jd_skills.replace(',','|').lower()
  for i in jd_skills.split(','):
    if '_' in i:
      underscore.append(i)
      mapper[i.lower().replace('_',' ')] = i
  jd_skills=jd_skills.replace(' ','_')
  jd_skills=jd_skills.replace(',',', ')
  for i in jd_skills.split(', '):
    if i not in underscore:
      if '_' in i:
        mapper[i.lower().replace('_',' ')] = i.replace('_',' ')
      elif '-' in i:
        mapper[i.lower().replace('-',' ')] = i
      else:
        mapper[i.lower()] = i
  jd_skills=jd_skills.replace('-','_')

  phrases=Phrases.load('/content/drive/MyDrive/Complete_Resume_Screening/Resume_Screening_Project/Phrases_split.pkl')
  w2v_model1 = joblib.load('/content/drive/MyDrive/Complete_Resume_Screening/Resume_Screening_Project/word2vec_model.model')


  lines = [preprocess(jd_skills.lower().rstrip())]
  final_jd_skills=list(set(lines[0]).intersection(skills['0']))

  print(jd_skills)






  res = check(pdftext, skills, lines[0], w2v_model1, phrases, pattern)

  res_dict={}
  for i in res.keys():
    j=i.replace('_',' ')
    res_dict[mapper[j]] = res[i]
  print('skills_matched :',res_dict)

  skills_weigths = {}

  f = open('/content/drive/MyDrive/Complete_Resume_Screening /Resume_Screening_Project/required_skills.txt','r')
  next(f)
  next(f)
  list_of_lines = f.readlines()
  f.close()

  skills_weigths = string_to_dict(list_of_lines)


  return calculate_weighted_similarity_score(res_dict, skills_weigths)



def substring_after(s, delim):
    return s.partition(delim)[2]

required_skills = """role : Software Developer
years of experience : 2
kaggle:5
ml:7
python:10
mysql:4
oracle:1
pytorch:8
scikit_learn:6
JavaScript:2
PHP:9
django:3"""

f = open('/content/drive/MyDrive/Complete_Resume_Screening/Resume_Screening_Project/required_skills.txt','w')
f.write(required_skills)
f.close()

f = open('/content/drive/MyDrive/Complete_Resume_Screening/Resume_Screening_Project/required_skills.txt','r')
role = f.readline()
role = role.strip()
years_of_experience = f.readline()
years_of_experience = years_of_experience.strip()
f.close()

role = substring_after(role, ': ')
years_of_experience = substring_after(years_of_experience, ': ')


compatible = []
for i in range(len(resume_strs)):
  get_candidate_compatability(required_skills,resume_strs[i])
  if get_candidate_compatability(required_skills,resume_strs[i]) >= 0.5:
    compatible.append(names[i])


candidate_accepted = []

with open('/content/drive/MyDrive/Complete_Resume_Screening /Resume_Screening_Project/candidates.csv', 'r') as csvfile:
  csv_reader = csv.reader(csvfile)
  for row in csv_reader:


    if (row[7]).strip("[]'").lower() ==role.lower() and row[10] == years_of_experience.lower() and row[0] in compatible:
      candidate_accepted.append(row[0])

    else:
      continue




#result_dict = skills_weights



for i in candidate_accepted:
  print("candidate accpeted: ",i)

role_Software_Developer, years_of_experience_, kaggle, ml, python, mysql, oracle, pytorch, scikit_learn, JavaScript, PHP, django
skills_matched : {'django': 1, 'mysql': 1, 'ml': 1, 'PHP': 1, 'python': 1, 'JavaScript': 1, 'years of experience ': 1, 'role Software Developer': 0, 'kaggle': 0, 'scikit_learn': 0, 'oracle': 0, 'pytorch': 0}
role_Software_Developer, years_of_experience_, kaggle, ml, python, mysql, oracle, pytorch, scikit_learn, JavaScript, PHP, django
skills_matched : {'django': 1, 'mysql': 1, 'ml': 1, 'PHP': 1, 'python': 1, 'JavaScript': 1, 'years of experience ': 1, 'role Software Developer': 0, 'kaggle': 0, 'scikit_learn': 0, 'oracle': 0, 'pytorch': 0}


role_Software_Developer, years_of_experience_, kaggle, ml, python, mysql, oracle, pytorch, scikit_learn, JavaScript, PHP, django
skills_matched : {'django': 1, 'mysql': 1, 'ml': 1, 'python': 1, 'JavaScript': 1, 'role Software Developer': 0, 'years of experience ': 0, 'PHP': 0, 'kaggle': 0, 'scikit_learn': 0, 'oracle': 0, 'pytorch': 0}
role_Software_Developer, years_of_experience_, kaggle, ml, python, mysql, oracle, pytorch, scikit_learn, JavaScript, PHP, django
skills_matched : {'django': 1, 'mysql': 1, 'ml': 1, 'python': 1, 'JavaScript': 1, 'role Software Developer': 0, 'years of experience ': 0, 'PHP': 0, 'kaggle': 0, 'scikit_learn': 0, 'oracle': 0, 'pytorch': 0}


role_Software_Developer, years_of_experience_, kaggle, ml, python, mysql, oracle, pytorch, scikit_learn, JavaScript, PHP, django
skills_matched : {'ml': 1, 'years of experience ': 1, 'django': 0, 'kaggle': 0, 'scikit_learn': 0, 'oracle': 0, 'mysql': 0, 'pytorch': 0, 'role Software Developer': 0, 'PHP': 0, 'python': 0, 'JavaScript': 0}
role_Software_Developer, years_of_experience_, kaggle, ml, python, mysql, oracle, pytorch, scikit_learn, JavaScript, PHP, django
skills_matched : {'ml': 1, 'years of experience ': 1, 'django': 0, 'kaggle': 0, 'scikit_learn': 0, 'oracle': 0, 'mysql': 0, 'pytorch': 0, 'role Software Developer': 0, 'PHP': 0, 'python': 0, 'JavaScript': 0}


role_Software_Developer, years_of_experience_, kaggle, ml, python, mysql, oracle, pytorch, scikit_learn, JavaScript, PHP, django
skills_matched : {'years of experience ': 0, 'django': 0, 'kaggle': 0, 'scikit_learn': 0, 'oracle': 0, 'mysql': 0, 'pytorch': 0, 'role Software Developer': 0, 'ml': 0, 'PHP': 0, 'python': 0, 'JavaScript': 0}
role_Software_Developer, years_of_experience_, kaggle, ml, python, mysql, oracle, pytorch, scikit_learn, JavaScript, PHP, django
skills_matched : {'years of experience ': 0, 'django': 0, 'kaggle': 0, 'scikit_learn': 0, 'oracle': 0, 'mysql': 0, 'pytorch': 0, 'role Software Developer': 0, 'ml': 0, 'PHP': 0, 'python': 0, 'JavaScript': 0}
role_Software_Developer, years_of_experience_, kaggle, ml, python, mysql, oracle, pytorch, scikit_learn, JavaScript, PHP, django
skills_matched : {'years of experience ': 0, 'django': 0, 'kaggle': 0, 'scikit_learn': 0, 'oracle': 0, 'mysql': 0, 'pytorch': 0, 'role Software Developer': 0, 'ml': 0, 'PHP': 0, 'python': 0, 'JavaS

role_Software_Developer, years_of_experience_, kaggle, ml, python, mysql, oracle, pytorch, scikit_learn, JavaScript, PHP, django
skills_matched : {'years of experience ': 0, 'django': 0, 'kaggle': 0, 'scikit_learn': 0, 'oracle': 0, 'mysql': 0, 'pytorch': 0, 'role Software Developer': 0, 'ml': 0, 'PHP': 0, 'python': 0, 'JavaScript': 0}
role_Software_Developer, years_of_experience_, kaggle, ml, python, mysql, oracle, pytorch, scikit_learn, JavaScript, PHP, django
skills_matched : {'django': 1, 'oracle': 1, 'mysql': 1, 'ml': 1, 'PHP': 1, 'python': 1, 'JavaScript': 1, 'role Software Developer': 0, 'years of experience ': 0, 'kaggle': 0, 'scikit_learn': 0, 'pytorch': 0}
role_Software_Developer, years_of_experience_, kaggle, ml, python, mysql, oracle, pytorch, scikit_learn, JavaScript, PHP, django
skills_matched : {'django': 1, 'oracle': 1, 'mysql': 1, 'ml': 1, 'PHP': 1, 'python': 1, 'JavaScript': 1, 'role Software Developer': 0, 'years of experience ': 0, 'kaggle': 0, 'scikit_learn': 0, 'py

In [ ]:
for i in candidate_accepted:
  print("candidate accpeted: ",i)

candidate accpeted:  Lalita Lalmani
